In [1]:
import datetime
import os
import pandas as pd
import pypyodbc

os.chdir("C:/Users/johnlhc/Desktop/volume_forecasting")

In [2]:
CONNECTION = pypyodbc.connect("Driver={SQL Server};"
                              "Server=op2app1;")

In [3]:
# mm/dd/YYYY
START_DT = "12/30/2019"
END_DT = "01/26/2020"

# Helper Functions

In [4]:
def convert_to_dt(df, col_list, dt_format):
    for col in col_list:
        df[col] = pd.to_datetime(df[col], format=dt_format)
        
    return df

def generate_csd(df, consortium, line, service, direction):
    df.loc[df[consortium].isna(), "CSD"] = (df.loc[df[consortium].isna(), line]
                                            + "-"
                                            + df.loc[df[consortium].isna(), service]
                                            + "-"
                                            + df.loc[df[consortium].isna(), direction])
    
    df.loc[~df[consortium].isna(), "CSD"] = (df.loc[~df[consortium].isna(), consortium]
                                             + "-"
                                             + df.loc[~df[consortium].isna(), service]
                                             + "-"
                                             + df.loc[~df[consortium].isna(), direction])
    
    return df

def define_week_num(df, psa_week, reference_dt_col):
    for i, week in psa_week.iterrows():
        start_dt = week.loc["start_date"]
        end_dt = week.loc["end_date"]
        week_num = week.loc["week"]

        df.loc[(df[reference_dt_col] >= start_dt) & 
               (df[reference_dt_col] <= end_dt), "week"] = week_num
    
    df["week"] = df["week"].astype(int)
    return df

def define_year(df, psa_week, reference_dt_col):
    for i, week in psa_week.iterrows():
        start_dt = week.loc["start_date"]
        end_dt = week.loc["end_date"]
        year = week.loc["year"]

        df.loc[(df[reference_dt_col] >= start_dt) & 
               (df[reference_dt_col] <= end_dt), "year"] = year
    
    df["year"] = df["year"].astype(int)
    return df

def strip_str_cols(df):
    df_obj_cols = df.select_dtypes("object").columns
    df[df_obj_cols] = df[df_obj_cols].apply(lambda x: x.str.strip())
    
    return df

def generate_general_csd(df):
    df["general_CSD"] = df["CSD"].replace("[1-9]", "", regex=True)
    
    return df
    
    

# Plot Function

# PSA Week

Get Data

In [5]:
psa_week_sql = "SELECT * FROM reportadmin.GLOBAL_PSA_WEEK \
WHERE START_DATE BETWEEN '12/01/2015' AND '06/01/2020'"#.format(START_DT, END_DT)

In [6]:
psa_week = pd.read_sql(psa_week_sql, CONNECTION)

Preprocess

In [7]:
psa_week["start_date"] = psa_week["start_date"] + datetime.timedelta(hours=0, minutes=0, seconds=0)
psa_week["end_date"] = psa_week["end_date"] + datetime.timedelta(hours=23, minutes=59, seconds=59)
psa_week = convert_to_dt(df=psa_week, col_list=["start_date", "end_date"], dt_format="%Y-%m-%d")
psa_week = strip_str_cols(df=psa_week)

# Vessel Utilization

Get Data

In [8]:
vsl_util_sql = "SELECT * FROM reportadmin.V_PD_VSL_UTILIZATION \
WHERE FIRST_ATB_DT BETWEEN '{}' AND '{}' and vessel_status_x = 'Departed'".format(START_DT, END_DT)

In [9]:
vsl_util = pd.read_sql(vsl_util_sql, CONNECTION)

Preprocess

In [10]:
vsl_util = convert_to_dt(df=vsl_util, col_list=["first_atb_dt", "last_atu_dt"], dt_format="%Y-%m-%d %H:%M:%S")
vsl_util = strip_str_cols(df=vsl_util)

Generate CSD

In [11]:
vsl_util = generate_csd(df=vsl_util,
                        consortium="consortium_c",
                        line="line_c",
                        service="service_c",
                        direction="direction_c")

Generate General CSD

In [12]:
vsl_util = generate_general_csd(df=vsl_util)

# Vsl Dtl

In [13]:
vsl_dtl_sql = "SELECT ABBR_VESSEL_M, ABBR_VOYAGE_OUT_N, ABBR_VOYAGE_IN_N, DIRECTION_C \
FROM reportadmin.V_PD_OPS_VSL_DTL \
WHERE ATB_DT BETWEEN '{}' AND '{}' AND (IGB_MODE_I = '' OR IGB_MODE_I Is Null)\
AND (SERVICE_C Is Not Null or SERVICE_C <> '')".format(START_DT, END_DT)

In [14]:
vsl_dtl = pd.read_sql(vsl_dtl_sql, CONNECTION)

Preprocess

In [15]:
vsl_dtl = strip_str_cols(df=vsl_dtl)

# Volume

<h3> Discharge </h3>

In [16]:
vol_sql_discharge = "SELECT * FROM reportadmin.V_PD_OPS_BIZ_TPUT_DTL \
WHERE DISC_ATB_DT BETWEEN '{}' AND '{}' AND EVENT_C = 'DISC'".format(START_DT, END_DT)

In [17]:
vol_discharge = pd.read_sql(vol_sql_discharge, CONNECTION)

Preprocess

In [18]:
vol_discharge = strip_str_cols(vol_discharge)

In [19]:
vol_discharge.loc[vol_discharge["disc_consortium_c"] == "", "disc_consortium_c"] = None

Get Direction

In [20]:
vol_discharge = vol_discharge.merge(vsl_dtl,
                                    how="left",
                                    left_on=["disc_abbr_vessel_m", "disc_abbr_voyage_in_n"],
                                    right_on=["abbr_vessel_m", "abbr_voyage_in_n"])

Generate CSD

In [21]:
vol_discharge = generate_csd(df=vol_discharge,
                             consortium="disc_consortium_c",
                             line="disc_line_c",
                             service="disc_service_c",
                             direction="direction_c")

Generate General CSD

In [22]:
vol_discharge = generate_general_csd(df=vol_discharge)

<h3> Load </h3>

In [23]:
vol_sql_load = "SELECT * FROM reportadmin.V_PD_OPS_BIZ_TPUT_DTL \
WHERE LOAD_ATB_DT BETWEEN '{}' AND '{}' AND EVENT_C = 'LOAD'".format(START_DT, END_DT)

In [24]:
vol_load = pd.read_sql(vol_sql_load, CONNECTION)

Preprocess

In [25]:
vol_load = strip_str_cols(vol_load)

In [26]:
vol_load.loc[vol_load["load_consortium_c"] == "", "load_consortium_c"] = None

Get Direction

In [27]:
vol_load = vol_load.merge(vsl_dtl,
                          how="left",
                          left_on=["load_abbr_vessel_m", "load_abbr_voyage_out_n"],
                          right_on=["abbr_vessel_m", "abbr_voyage_out_n"])

 Generate CSD

In [28]:
vol_load = generate_csd(df=vol_load,
                        consortium="load_consortium_c",
                        line="load_line_c",
                        service="load_service_c",
                        direction="direction_c")

Generate General CSD

In [29]:
vol_load = generate_general_csd(df=vol_load)

# Define PSA Week

In [30]:
vsl_util = define_week_num(df=vsl_util, psa_week=psa_week, reference_dt_col="first_atb_dt")
vsl_util = define_year(df=vsl_util, psa_week=psa_week, reference_dt_col="first_atb_dt")

In [31]:
vol_discharge = define_week_num(df=vol_discharge, psa_week=psa_week, reference_dt_col="disc_atb_dt")
vol_discharge = define_year(df=vol_discharge, psa_week=psa_week, reference_dt_col="disc_atb_dt")

In [32]:
vol_load = define_week_num(df=vol_load, psa_week=psa_week, reference_dt_col="load_atb_dt")
vol_load = define_year(df=vol_load, psa_week=psa_week, reference_dt_col="load_atb_dt")

In [33]:
vol_load.columns

Index(['event_d', 'event_hour_t', 'event_c', 'disc_abbr_vessel_m',
       'disc_abbr_voyage_in_n', 'disc_berth_seq_n', 'disc_atb_dt',
       'disc_tsa_ref_c', 'disc_derived_atb_d', 'disc_berth_m', 'disc_line_c',
       'disc_consortium_c', 'disc_service_c', 'disc_slot_opr_c',
       'load_abbr_vessel_m', 'load_abbr_voyage_out_n', 'load_berth_seq_n',
       'load_atb_dt', 'load_tsa_ref_c', 'load_derived_atb_d', 'load_berth_m',
       'load_line_c', 'load_consortium_c', 'load_service_c', 'load_slot_opr_c',
       'cntr_opr_c', 'load_port_c', 'disc_port1_c', 'disc_port2_c',
       'disc_port3_c', 'dest_port_c', 'derived_cntr_purp_c',
       'derived_cntr_type_c', 'length_q', 'cntr_st_c', 'cat_c', 'cntr_type_c',
       'special_detail_x', 'dg_group_c', 'dg_imo_class_c', 'reefer_temp_q',
       'rent_i', 'over_size_i', 'uc_i', 'berth_opr_org_c', 'deviate_reason_c',
       'bill_op_i', 'batch_id', 'box_q', 'teu_q', 'abbr_vessel_m',
       'abbr_voyage_out_n', 'abbr_voyage_in_n', 'direction_c

In [34]:
vol_load.groupby(["week", "dest_port_c"]).agg({"teu_q": "sum"}).reset_index().to_csv("pod.csv", index=False)

# Group by CSD

Vsl Util

In [35]:
csd_util = vsl_util.groupby(["CSD", "year", "week"]).agg({"vv_c": "count",
                                                          "vsl_capacity_teu_q": "sum",
                                                          "inbound_teu_q": "sum",
                                                          "outbound_teu_q": "sum"}).reset_index(drop=False)

In [36]:
csd_util.loc[:, "inbound_util_p"] = csd_util.loc[:, "inbound_teu_q"] / csd_util.loc[:, "vsl_capacity_teu_q"]
csd_util.loc[:, "outbound_util_p"] = csd_util.loc[:, "outbound_teu_q"] / csd_util.loc[:, "vsl_capacity_teu_q"]

In [37]:
csd_util = csd_util.rename(columns={"vv_c": "num_calls"})

Possible:

<ul> Container type </ul>
<ul> Load port </ul>
<ul> Disc port </ul>

Volume -- Discharge

In [38]:
csd_disc = vol_discharge.groupby(["CSD", "cntr_st_c", "year", "week"]).agg({"disc_abbr_vessel_m": "nunique",
                                                                            "teu_q": "sum"}).reset_index(drop=False)

In [39]:
csd_disc = csd_disc.rename(columns={"disc_abbr_vessel_m": "num_calls"})

Volume -- Load

In [40]:
csd_load = vol_load.groupby(["CSD", "cntr_st_c", "year", "week"]).agg({"load_abbr_vessel_m": "nunique",
                                                                       "teu_q": "sum"}).reset_index(drop=False)

In [41]:
csd_load = csd_load.rename(columns={"load_abbr_vessel_m": "num_calls"})

# Sort Values

In [42]:
csd_util = csd_util.sort_values(by=["week"], ascending=True)
csd_disc = csd_disc.sort_values(by=["week"], ascending=True)
csd_load = csd_load.sort_values(by=["week"], ascending=True)

# EU2-W

In [43]:
util_eu2_w = csd_util[csd_util["CSD"].str.contains("EU2-W")]
vol_disc_eu2_w = csd_disc[csd_disc["CSD"].str.contains("EU2-W")]
vol_load_eu2_w = csd_load[csd_load["CSD"].str.contains("EU2-W")]

Plotting

plot_chart(vol=vol_load_eu2_w, util=util_eu2_w, service="EU2-W", discharge=False)

# Group By General CSD

Vsl Util

In [44]:
general_csd_util = vsl_util.groupby(["general_CSD", "year", "week"]).agg({"vv_c": "count",
                                                                          "vsl_capacity_teu_q": "sum",
                                                                          "inbound_teu_q": "sum",
                                                                          "outbound_teu_q": "sum"}).reset_index(drop=False)

general_csd_util.loc[:, "inbound_util_p"] = general_csd_util.loc[:, "inbound_teu_q"] / general_csd_util.loc[:, "vsl_capacity_teu_q"]
general_csd_util.loc[:, "outbound_util_p"] = general_csd_util.loc[:, "outbound_teu_q"] / general_csd_util.loc[:, "vsl_capacity_teu_q"]

general_csd_util = general_csd_util.rename(columns={"vv_c": "num_calls"})

general_csd_util = general_csd_util.sort_values(by=["week"], ascending=True)

Volume Discharge

In [45]:
general_csd_disc = vol_discharge.groupby(["general_CSD", "cntr_st_c", "year", "week"]).agg({"disc_abbr_vessel_m": "nunique",
                                                                                            "teu_q": "sum"}).reset_index(drop=False)

general_csd_disc = general_csd_disc.rename(columns={"disc_abbr_vessel_m": "num_calls"})

general_csd_disc = general_csd_disc.sort_values(by=["week"], ascending=True)

Volume Load

In [46]:
general_csd_load = vol_load.groupby(["general_CSD", "cntr_st_c", "year", "week"]).agg({"load_abbr_vessel_m": "nunique",
                                                                                       "teu_q": "sum"}).reset_index(drop=False)

general_csd_load = general_csd_load.rename(columns={"load_abbr_vessel_m": "num_calls"})

general_csd_load = general_csd_load.sort_values(by=["week"], ascending=True)

# EU-W

In [47]:
util_eu_w = general_csd_util[general_csd_util["general_CSD"].str.contains("EU-W")]
vol_disc_eu_w = general_csd_disc[general_csd_disc["general_CSD"].str.contains("EU-W")]
vol_load_eu_w = general_csd_load[general_csd_load["general_CSD"].str.contains("EU-W")]

plot_chart(vol=vol_load_eu_w, util=util_eu_w, service="EU-W", discharge=False)

Save to CSV

In [48]:
util_eu_w.to_csv("util_eu_w.csv", index=False)
vol_disc_eu_w.to_csv("vol_disc_eu_w.csv", index=False)
vol_load_eu_w.to_csv("vol_load_eu_w.csv", index=False)

# ME-W

util_me_w = general_csd_util[general_csd_util["general_CSD"].str.contains("-ME-W")]
vol_disc_me_w = general_csd_disc[general_csd_disc["general_CSD"].str.contains("-ME-W")]
vol_load_me_w = general_csd_load[general_csd_load["general_CSD"].str.contains("-ME-W")]

util_me_w

plot_chart(vol=vol_load_me_w, util=util_me_w, service="ME-W", discharge=False)

Save to CSV

util_me_w[util_me_w.year == 2019].to_csv("util_me_w.csv", index=False)
vol_disc_me_w[vol_disc_me_w.year == 2019].to_csv("vol_disc_me_w.csv", index=False)
vol_load_me_w[vol_load_me_w.year == 2019].to_csv("vol_load_me_w.csv", index=False)

# FE-W

In [49]:
util_fe_w = general_csd_util[general_csd_util["general_CSD"].str.contains("-FE-W")]
vol_disc_fe_w = general_csd_disc[general_csd_disc["general_CSD"].str.contains("-FE-W")]
vol_load_fe_w = general_csd_load[general_csd_load["general_CSD"].str.contains("-FE-W")]

plot_chart(vol=vol_load_fe_w, util=util_fe_w, service="FE-W", discharge=False)

In [50]:
util_fe_w.to_csv("util_fe_w.csv", index=False)
vol_disc_fe_w.to_csv("vol_disc_fe_w.csv", index=False)
vol_load_fe_w.to_csv("vol_load_fe_w.csv", index=False)

# EU-E

util_eu_e = general_csd_util[general_csd_util["general_CSD"].str.contains("-EU-E")]
vol_disc_eu_e = general_csd_disc[general_csd_disc["general_CSD"].str.contains("-EU-E")]
vol_load_eu_e = general_csd_load[general_csd_load["general_CSD"].str.contains("-EU-E")]

plot_chart(vol=vol_load_eu_e, util=util_eu_e, service="EU-E", discharge=False)

util_eu_e